In [1]:
!pip install datasets librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 4.1 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.8.4.1 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cufft-cu12==11.2.1.3; platform_system == "Linux" and platform_machine == "x86_64",

In [2]:
import argparse
import math
import os
import random
from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Union

import datasets
import torch
from accelerate import Accelerator
from accelerate.logging import get_logger
from datasets import DatasetDict, concatenate_datasets, load_dataset
from huggingface_hub import HfApi
from torch.utils.data.dataloader import DataLoader
from tqdm.auto import tqdm

import transformers
from transformers import (
    SchedulerType,
    Wav2Vec2Config,
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForPreTraining,
    get_scheduler,
    is_wandb_available,
    set_seed,
)
from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices
from transformers.utils import send_example_telemetry


logger = get_logger(__name__)

2025-05-11 16:54:58.200676: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746982498.356326      31 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746982498.400516      31 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [27]:
from dataclasses import dataclass, field
from transformers import SchedulerType
import os

@dataclass
class Config:
    dataset_dir: str = "/kaggle/input/wav2vec2-0pret/Dataset"        # ✅ Kaggle dataset path
    checkpoint_dir: str = "/kaggle/working/TrainingCheckpoints/training_state.pt"       # ✅ Kaggle writable dir
    load_config_path : str = "/kaggle/working/TrainingCheckpoints/iter_index.txt"
    bengali_tokenizer: str = "tanmoyio/wav2vec2-large-xlsr-bengali"
    epoch_per_chunk: int = 1
    model_name_or_path: str = "patrickvonplaten/wav2vec2-base-v2" # Facebook/wav2vec2
    trust_remote_code: bool = True
    preprocessing_num_workers: int = None
    overwrite_cache: bool = False
    preprocessing_only: bool = False
    cache_dir: str = None
    validation_split_percentage: int = 20
    logging_steps: int = 1
    saving_steps: int = 10000
    audio_column_name: str = "audio"
    config_name: str = None
    train_cache_file_name: str = None
    validation_cache_file_name: str = None
    per_device_train_batch_size: int = 8
    per_device_eval_batch_size: int = 8
    learning_rate: float = 0.001
    weight_decay: float = 0.01
    max_train_steps: int = 32
    gradient_accumulation_steps: int = 4
    gradient_checkpointing: bool = True
    lr_scheduler_type: SchedulerType = "linear"
    num_warmup_steps: int = 32000
    output_dir: str = "./wav2vec2-pretrained-demo"
    seed: int = 0
    max_gumbel_temperature: float = 2.0
    min_gumbel_temperature: float = 0.5
    gumbel_temperature_decay: float = 0.999995
    max_duration_in_seconds: float = 20.0
    min_duration_in_seconds: float = 2.0
    pad_to_multiple_of: int = None
    adam_beta1: float = 0.9
    adam_beta2: float = 0.98
    adam_epsilon: float = 1e-6
    push_to_hub: bool = False
    hub_model_id: str = None
    hub_token: str = None
    mask_time_prob: float = 0.65
    mask_time_length: int = 10


config = Config()

In [309]:
# Define the path to the iter_index.txt file in a writable directory
iter_index_path = "/kaggle/working/TrainingCheckpoints/iter_index.txt"

# Check if the file exists and read the contents
if os.path.exists(iter_index_path):
    with open(iter_index_path, "r") as f:
        iter_index = f.read().strip()  # Read and strip any extra spaces or newlines
    print(f"Contents of iter_index.txt: {iter_index}")
else:
    print(f"ℹ️ [LOAD] iter_index.txt not found at {iter_index_path}")


Contents of iter_index.txt: 35


In [310]:
if os.path.exists(config.load_config_path):
    with open(config.load_config_path, "r") as f:
        iter_index = int(f.read().strip())
else:
    iter_index = 0

print(f"❇️ [LOAD INFO] Iteration Index: {iter_index}")

❇️ [LOAD INFO] Iteration Index: 35


In [311]:
from dataclasses import dataclass
from typing import Optional, Union
import torch
from transformers import Wav2Vec2ForPreTraining, Wav2Vec2FeatureExtractor
from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices


@dataclass
class DataCollatorForWav2Vec2Pretraining:
    """
    Data collator for dynamic padding and generating masked indices for wav2vec2 pretraining.
    """
    model: Wav2Vec2ForPreTraining
    feature_extractor: Wav2Vec2FeatureExtractor
    config: Config  # <- use config for relevant parameters

    def __call__(self, features: list[dict[str, Union[list[int], torch.Tensor]]]) -> dict[str, torch.Tensor]:
        # reformat list to dict and set to pytorch format
        batch = self.feature_extractor.pad(
            features,
            padding="longest",  # From original class; can also be `self.config.padding_strategy` if desired
            pad_to_multiple_of=self.config.pad_to_multiple_of,
            return_tensors="pt",
        )

        device = batch["input_values"].device
        batch_size = batch["input_values"].shape[0]

        mask_indices_seq_length = self.model._get_feat_extract_output_lengths(batch["input_values"].shape[-1])
        mask_indices_seq_length = int(mask_indices_seq_length)

        # print(f"Inside data collator. Mask_time_prob: {self.config.mask_time_prob}")

        if batch.get("attention_mask") is not None:
            batch["sub_attention_mask"] = self.model._get_feature_vector_attention_mask(
                mask_indices_seq_length, batch["attention_mask"]
            )

        features_shape = (batch_size, mask_indices_seq_length)

        mask_time_indices = _compute_mask_indices(
            features_shape,
            self.config.mask_time_prob,
            self.config.mask_time_length,
            attention_mask=batch.get("sub_attention_mask"),
        )

        sampled_negative_indices = _sample_negative_indices(
            features_shape,
            self.model.config.num_negatives,
            mask_time_indices=mask_time_indices,
        )

        batch["mask_time_indices"] = torch.tensor(mask_time_indices, dtype=torch.long, device=device)
        batch["sampled_negative_indices"] = torch.tensor(sampled_negative_indices, dtype=torch.long, device=device)

        return batch

In [312]:
def multiply_grads(params, c):
    """Multiplies grads by a constant *c*."""
    for p in params:
        if p.grad is not None:
            if torch.is_tensor(c):
                c = c.to(p.grad.device)
            p.grad.data.mul_(c)


def get_grad_norm(params, scale=1):
    """Compute grad norm given a gradient scale."""
    total_norm = 0.0
    for p in params:
        if p.grad is not None:
            param_norm = (p.grad.detach().data / scale).norm(2)
            total_norm += param_norm.item() ** 2
    total_norm = total_norm**0.5
    return total_norm

In [313]:
import os
import logging
from transformers import is_wandb_available
from accelerate import Accelerator

# Initialize the Accelerator
accelerator = Accelerator()

# Setup logger
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

# W&B logic
if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_info()
    if 'KAGGLE_KERNEL_RUN_TYPE' not in os.environ and is_wandb_available():  # Check if running outside Kaggle
        try:
            import wandb
            # Initialize W&B in offline mode (logs locally without needing internet)
            wandb.init(project=config.output_dir.split("/")[-1], mode="offline")
            logger.info("W&B initialized in offline mode.")
        except Exception as e:
            logger.error(f"Error initializing W&B: {e}")
    else:
        logger.info("ℹ️ [INFO] W&B is not available or skipping W&B in Kaggle. Using local logging.")
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()

# Check if the seed is set in the config and set it
if config.seed is not None:
    set_seed(config.seed)

# Check and create output directory if necessary
if accelerator.is_main_process:
    if config.push_to_hub and not config.preprocessing_only:
        repo_name = config.hub_model_id or Path(config.output_dir).absolute().name
        api = HfApi()
        repo_id = api.create_repo(repo_name, exist_ok=True, token=config.hub_token).repo_id
        with open(os.path.join(config.output_dir, ".gitignore"), "w+") as gitignore:
            if "step_*" not in gitignore:
                gitignore.write("step_*\n")
            if "epoch_*" not in gitignore:
                gitignore.write("epoch_*\n")
    elif config.output_dir is not None:
        os.makedirs(config.output_dir, exist_ok=True)

accelerator.wait_for_everyone()

# Rest of your code continues here...


In [314]:
chunk_path = f"{config.dataset_dir}/dataset_chunk_{iter_index}.parquet"

assert os.path.exists(chunk_path), f"Missing dataset chunk: {chunk_path}"
dataset = load_dataset("parquet", data_files={'train': chunk_path})
print(f"❇️ [LOAD INFO] Loaded data from {chunk_path}")


dataset = dataset.remove_columns(['path', 'labels'])

# Split into train and validation
train_valid_split = dataset['train'].train_test_split(test_size=187, shuffle=True, seed=42)

dataset = DatasetDict({
    'train': train_valid_split['train'],
    'validation': train_valid_split['test']
})

train_dataset = dataset['train']
eval_dataset = dataset['validation']

Generating train split: 0 examples [00:00, ? examples/s]

❇️ [LOAD INFO] Loaded data from /kaggle/input/wav2vec2-0pret/Dataset/dataset_chunk_35.parquet


In [315]:
from datasets import load_dataset, DatasetDict
import os, json
import numpy as np
from transformers import TrainingArguments, Trainer
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2CTCTokenizer, Wav2Vec2ForCTC

import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

In [316]:
from transformers import Wav2Vec2CTCTokenizer

# Specify the local path where the tokenizer files are saved
local_tokenizer_path = "/kaggle/input/tanmoyio-wav2vec2-0"  # Replace with actual path

# Load the tokenizer from the local directory
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(local_tokenizer_path)

# Verify tokenizer loaded correctly
print("Tokenizer loaded successfully.")


loading file vocab.json
loading file tokenizer_config.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer.json
loading file chat_template.jinja
loading configuration file /kaggle/input/tanmoyio-wav2vec2-0/config.json
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": true,
  "diversity_loss_weight": 0.1,
  "do_stable

Tokenizer loaded successfully.


In [317]:
tokenizer = Wav2Vec2CTCTokenizer.from_pretrained(
    "/kaggle/input/tanmoyio-wav2vec2-0",  # Local path to the tokenizer directory
    unk_token="[UNK]",
    pad_token="[PAD]",
    word_delimiter_token="|"
)

loading file vocab.json
loading file tokenizer_config.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer.json
loading file chat_template.jinja
loading configuration file /kaggle/input/tanmoyio-wav2vec2-0/config.json
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "mean",
  "ctc_zero_infinity": true,
  "diversity_loss_weight": 0.1,
  "do_stable

In [318]:
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained("/kaggle/input/tanmoyio-wav2vec2-0")

if not feature_extractor.do_normalize:
    raise ValueError("Feature extractor must normalize inputs (do_normalize=True).")


loading configuration file /kaggle/input/tanmoyio-wav2vec2-0/preprocessor_config.json
Feature extractor Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0.0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}



In [319]:
# Import the necessary class
from transformers import Wav2Vec2Config

# Load the model config from the local path or Hugging Face model
model_config = Wav2Vec2Config.from_pretrained("/kaggle/input/wav2vec2-base-v2")

# Update vocab_size in the config
model_config.vocab_size = tokenizer.vocab_size  # Set the tokenizer's vocab_size in the config

print("Model config loaded and vocab_size updated.")


loading configuration file /kaggle/input/wav2vec2-base-v2/config.json
Model config Wav2Vec2Config {
  "activation_dropout": 0.0,
  "adapter_attn_dim": null,
  "adapter_kernel_size": 3,
  "adapter_stride": 2,
  "add_adapter": false,
  "apply_spec_augment": true,
  "architectures": [
    "Wav2Vec2ForPreTraining"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "codevector_dim": 256,
  "contrastive_logits_temperature": 0.1,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_dropout": 0.0,
  "feat_extract_norm": "layer",
  "feat_proj_dropout": 0.0,
  "feat_quanti

Model config loaded and vocab_size updated.


In [320]:
if not model_config.do_stable_layer_norm or model_config.feat_extract_norm != "layer":
    raise ValueError("Only supports stable layer norm with `feat_extract_norm='layer'`")

model = Wav2Vec2ForPreTraining(model_config)

if config.gradient_checkpointing:
    model.gradient_checkpointing_enable()

mask_time_prob = config.mask_time_prob or model_config.mask_time_prob
mask_time_length = config.mask_time_length or model_config.mask_time_length

In [321]:
data_collator = DataCollatorForWav2Vec2Pretraining(
    model=model,
    feature_extractor=feature_extractor,
    config = config
)

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn=data_collator,
    batch_size=config.per_device_train_batch_size,
)
eval_dataloader = DataLoader(
    eval_dataset,
    collate_fn=data_collator,
    batch_size=config.per_device_eval_batch_size,
)

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=config.learning_rate,
    betas=(config.adam_beta1, config.adam_beta2),
    eps=config.adam_epsilon,
)

In [322]:
num_training_steps = config.max_train_steps
lr_scheduler = get_scheduler(
    config.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=100,
    num_training_steps=num_training_steps,
)

accelerator.register_for_checkpointing(lr_scheduler)

total_batch_size = config.per_device_train_batch_size * accelerator.num_processes * config.gradient_accumulation_steps
logger.info(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
logger.info(f"  Total optimization steps = {config.max_train_steps}")

In [323]:
if iter_index > 0:
    print("❇️ [LOAD INFO] LOADING STATE FROM SAVED FILES")
    load_path = os.path.join("/kaggle/working/TrainingCheckpoints", "training_state.pt")
    state = torch.load(load_path, map_location="cpu", weights_only=False)

    model.load_state_dict(state["model"])
    optimizer.load_state_dict(state["optimizer"])
    lr_scheduler.load_state_dict(state["scheduler"])

    if state.get("scaler") and hasattr(accelerator, "scaler"):
        accelerator.scaler.load_state_dict(state["scaler"])

    epoch = state["epoch"]
    completed_steps = state["completed_steps"]

    # Restore RNG states
    random.setstate(state["rng_state"]["random"])
    torch.set_rng_state(state["rng_state"]["torch"])
    if torch.cuda.is_available() and state["rng_state"]["cuda"]:
        torch.cuda.set_rng_state_all(state["rng_state"]["cuda"])

❇️ [LOAD INFO] LOADING STATE FROM SAVED FILES


In [324]:
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [325]:
# Initialize trackers
experiment_config = vars(config)
accelerator.init_trackers("wav2vec2_pretrain", experiment_config)

In [326]:
repo_id = "wav2vec_bengali_asr_pretraining"

In [ ]:
from tqdm import tqdm
import torch
import numpy as np

# Initialize progress bar
progress_bar = tqdm(range(config.max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0

for epoch in range(config.epoch_per_chunk):
    model.train()
    for step, batch in enumerate(train_dataloader):
        features_shape = batch["input_values"].shape  # [batch_size, raw_audio_len]
        print(f"📦 Batch shape: {features_shape}, num_negatives: {model.config.num_negatives}")

        # 🔍 Estimate downsampled feature length
        with torch.no_grad():
            downsampled_len = model._get_feat_extract_output_lengths(torch.tensor(features_shape[1]))

        # Skip the batch if downsampled_len is invalid (too small for negative sampling)
        if downsampled_len <= model.config.num_negatives:
            print(f"⚠️ Skipping batch: Downsampled feature length ({downsampled_len}) <= num_negatives ({model.config.num_negatives})")
            continue

        # ✅ Skip if no masked time indices
        num_losses = batch["mask_time_indices"].sum()
        if num_losses == 0:
            print("⚠️ Skipping batch due to no masked time indices.")
            continue

        # Handle sub_attention_mask if provided
        sub_attention_mask = batch.pop("sub_attention_mask", None)
        sub_attention_mask = sub_attention_mask if sub_attention_mask is not None else torch.ones_like(batch["mask_time_indices"])

        try:
            # ✅ Forward pass
            outputs = model(**batch)
        except ValueError as e:
            if "high <= 0" in str(e):
                print(f"🚨 Critical error in negative sampling - downsampled_len: {downsampled_len}, num_negatives: {model.config.num_negatives}")
                print("Skipping problematic batch...")
                continue
            raise  # Re-raise unexpected errors

        loss = outputs.loss / config.gradient_accumulation_steps
        accelerator.backward(loss)

        # ✅ Optimizer Step
        if (step + 1) % config.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            optimizer.zero_grad()

            if not accelerator.optimizer_step_was_skipped:
                lr_scheduler.step()
            else:
                print("⚠️ Skipping optimizer step due to overflow...")

            progress_bar.update(1)
            completed_steps += 1

        # Log training information
        if accelerator.is_local_main_process and (step + 1) % (config.gradient_accumulation_steps * config.logging_steps) == 0:
            loss_value = (loss * config.gradient_accumulation_steps) / batch["mask_time_indices"].sum()
            train_logs = {
                "loss": loss_value.item(),
                "%_mask_idx": (num_losses / sub_attention_mask.sum()).item(),
            }
            print(f"📢 Training Logs: {train_logs}")
            
        # Save the model periodically
        if (step + 1) % (config.gradient_accumulation_steps * config.saving_steps) == 0:
            if config.output_dir is not None:
                unwrapped_model = accelerator.unwrap_model(model)
                unwrapped_model.save_pretrained(config.output_dir, is_main_process=accelerator.is_main_process)

        # Stop training if max steps are reached
        if completed_steps >= config.max_train_steps:
            break

    # Validation Phase
    model.eval()
    val_logs = {"val_loss": 0, "val_num_losses": 0}
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            batch.pop("sub_attention_mask", None)
            outputs = model(**batch)

        val_logs["val_loss"] += outputs.loss.item()
        val_logs["val_num_losses"] += batch["mask_time_indices"].sum().item()

    # Summing over devices in multi-processing if needed
    if accelerator.num_processes > 1:
        val_logs = {k: accelerator.gather_for_metrics(v).sum() for k, v in val_logs.items()}

    avg_val_loss = val_logs["val_loss"] / val_logs["val_num_losses"] if val_logs["val_num_losses"] > 0 else float('nan')
    print(f"📢 Validation Logs: {{'val_loss': {avg_val_loss:.4f}}}")

    # Save model after validation
    if config.output_dir:
        unwrapped_model = accelerator.unwrap_model(model)
        unwrapped_model.save_pretrained(config.output_dir, is_main_process=accelerator.is_main_process)

# Final report
print(f"\n🎉 Training completed!")
print(f"Total completed steps: {completed_steps}")



100%|██████████| 32/32 [08:27<00:00, 15.86s/it]


📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100
📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100
📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100
📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100



  3%|▎         | 1/32 [00:12<06:42, 12.98s/it]

📢 Training Logs: {'loss': 0.09968750178813934, '%_mask_idx': 0.5126676559448242}
📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100
📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100
📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100
📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100



  6%|▋         | 2/32 [00:24<06:10, 12.36s/it]

...

📢 Training Logs: {'loss': 0.09968750178813934, '%_mask_idx': 0.509144127368927}
📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100
📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100
📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100
📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100



 84%|████████▍ | 27/32 [05:19<00:58, 11.77s/it]

📢 Training Logs: {'loss': 0.09968750178813934, '%_mask_idx': 0.5167785286903381}
📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100
📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100
📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100
📦 Batch shape: torch.Size([8, 160000]), num_negatives: 100



 88%|████████▊ | 28/32 [05:31<00:47, 11.79s/it]

📢 Training Logs: {'loss': 0.09968750178813934, '%_mask_idx': 0.506373941898346}


#### Save training states

In [308]:
import os

# ✅ Set Kaggle working directory for saving checkpoints and iteration index
save_path = "/kaggle/working/TrainingCheckpoints"
os.makedirs(save_path, exist_ok=True)

# ✅ Save the full training state (model, optimizer, scheduler, etc.)
state = {
    "epoch": config.epoch_per_chunk,
    "completed_steps": completed_steps,
    "model": accelerator.unwrap_model(model).state_dict(),
    "optimizer": optimizer.state_dict(),
    "scheduler": lr_scheduler.state_dict(),
    "scaler": accelerator.scaler.state_dict() if hasattr(accelerator, "scaler") and accelerator.scaler else None,
    "config": config,
    "rng_state": {
        "random": random.getstate(),
        "torch": torch.get_rng_state(),
        "cuda": torch.cuda.get_rng_state_all() if torch.cuda.is_available() else None,
    }
}

# ✅ Save checkpoint to the Kaggle working directory
accelerator.save(state, os.path.join(save_path, "training_state.pt"))

# ✅ Save the model to the save_path directory
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(save_path)

# ✅ Save the iteration index to track training progress
with open(os.path.join(save_path, "iter_index.txt"), "w") as f:
    f.write(str(iter_index + 1))

print(f"❇️ [LOAD INFO] Finished training on chunk {iter_index + 1}. Ready for next chunk.")
print(f"✅ Saved checkpoint, model, and iter_index to: {save_path}")


Configuration saved in /kaggle/working/TrainingCheckpoints/config.json
Model weights saved in /kaggle/working/TrainingCheckpoints/model.safetensors


❇️ [LOAD INFO] Finished training on chunk 35. Ready for next chunk.
✅ Saved checkpoint, model, and iter_index to: /kaggle/working/TrainingCheckpoints


In [150]:
import shutil
from IPython.display import FileLink

# Step 1: Zip the contents of the directory
shutil.make_archive('/kaggle/working/all_files', 'zip', '/kaggle/working/')

# Step 2: Provide a download link
FileLink('/kaggle/working/all_files.zip')


/kaggle/working/all_files.zip